In [1]:
from ase.io import read
from ase.visualize import view
from ase.neighborlist import natural_cutoffs, NeighborList
from scipy import sparse
import numpy as np

In [2]:
atoms = read('ttf-tcnq.cif')

In [3]:
nl_linkers = NeighborList(natural_cutoffs(atoms), self_interaction=False, bothways=True)
nl_linkers.update(atoms)
matrix = nl_linkers.get_connectivity_matrix(nl_linkers.nl)
n_components, component_list = sparse.csgraph.connected_components(matrix)
idx = 1
molIdx = component_list[idx]
print("There are {} molecules in the system".format(n_components))
print("Atom {} is part of molecule {}".format(idx, molIdx))
molIdxs = [ i for i in range(len(component_list)) if component_list[i] == molIdx ]
print("The following atoms are part of molecule {}: {}".format(molIdx, molIdxs))


There are 4 molecules in the system
Atom 1 is part of molecule 1
The following atoms are part of molecule 1: [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27]


In [4]:
edges = list(matrix.keys())

In [5]:
def find_min_config(min_pos, max_pos, distance, cell, index):
    if np.linalg.norm(max_pos - (min_pos + cell[index])) < distance:
        min_pos = min_pos + cell[index]
    return min_pos

In [6]:
max_bond_len = max(natural_cutoffs(atoms))
cell = list(atoms.get_cell())
new_atoms = read('ttf-tcnq.cif')
new_atoms.set_pbc([False,False,False])
all_positions = new_atoms.get_positions()
is_optimized = False
# For each bond, take the lower left and move it upper right until the bond shrinks
while not is_optimized:
    is_optimized = True
    for i in range(3):
        for edge in edges:
            positions = all_positions
            distance = np.linalg.norm(positions[edge[0]]-positions[edge[1]])
            if distance > max_bond_len*2:
                is_optimized = False
                min_pos = positions[edge[0]] if positions[edge[0],i] < positions[edge[1],i] else positions[edge[1]]
                max_pos = positions[edge[0]] if positions[edge[0],i] >= positions[edge[1],i] else positions[edge[1]]
                new_pos = find_min_config(min_pos, max_pos, distance, cell, i)
                if np.array_equal(min_pos,positions[edge[0]]):
                    all_positions[edge[0]] = new_pos
                    all_positions[edge[1]] = max_pos
                else:
                    all_positions[edge[0]] = max_pos
                    all_positions[edge[1]] = new_pos
new_atoms.set_positions(all_positions)
view(new_atoms)

In [7]:
from ase import Atoms
new_atoms = 